In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\Text-Summerization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\Text-Summerization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
#configurations for data transformation
from textSummaizer.constants import *
from textSummaizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
            
        
        
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation
        
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config
        

In [8]:
import os
from textSummaizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

c:\Users\ARSH\Desktop\MLOps\Text-Summerization\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
        logger.info(f"Tokenizer loaded from {self.config.tokenizer_name}")

    
    
    def convert_example_to_features(self,example_batch):
        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            truncation=True,
            padding='max_length',
            max_length=1024,
        )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'],
                truncation=True,
                padding='max_length',
                max_length=128,
            )
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
        
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt= dataset_samsum.map(
            self.convert_example_to_features,
            batched=True,
        )
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
        

In [10]:
#pipeline
try:
    config= ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    logger.exception(e)
    raise e



[2025-07-29 20:02:05,916 - textSummarizerlogger - INFO - YAML file config\config.yaml read successfully.]
[2025-07-29 20:02:05,916 - textSummarizerlogger - INFO - YAML file params.yaml read successfully.]
[2025-07-29 20:02:05,916 - textSummarizerlogger - INFO - Created directory: artifacts]
[2025-07-29 20:02:05,916 - textSummarizerlogger - INFO - Created directory: artifacts/data_transformation]
[2025-07-29 20:02:07,453 - textSummarizerlogger - INFO - Tokenizer loaded from t5-small]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\ARSH\Desktop\MLOps\Text-Summerization\.venv\lib\site-packages\transformers\tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

[2025-07-29 20:02:12,000 - textSummarizerlogger - ERROR - [Errno 22] Invalid argument: 'c:/Users/ARSH/Desktop/MLOps/Text-Summerization/artifacts/data_transformation/samsum_dataset/train/data-00000-of-00001.arrow']
Traceback (most recent call last):
  File "C:\Users\ARSH\AppData\Local\Temp\ipykernel_11524\1664774545.py", line 6, in <module>
    data_transformation.convert()
  File "C:\Users\ARSH\AppData\Local\Temp\ipykernel_11524\1656508627.py", line 36, in convert
    dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
  File "c:\Users\ARSH\Desktop\MLOps\Text-Summerization\.venv\lib\site-packages\datasets\dataset_dict.py", line 1350, in save_to_disk
    dataset.save_to_disk(
  File "c:\Users\ARSH\Desktop\MLOps\Text-Summerization\.venv\lib\site-packages\datasets\arrow_dataset.py", line 1626, in save_to_disk
    for job_id, done, content in Dataset._save_to_disk_single(**kwargs):
  File "c:\Users\ARSH\Desktop\MLOps\Text-Summerization\.venv\lib\site-packag

OSError: [Errno 22] Invalid argument: 'c:/Users/ARSH/Desktop/MLOps/Text-Summerization/artifacts/data_transformation/samsum_dataset/train/data-00000-of-00001.arrow'